In [20]:
from bs4 import BeautifulSoup

from urllib.request import urlopen

import re

import pandas as pd
import datetime

In [ ]:
def a_from_date(date):
    try:
        url = urlopen('http://www.saude.rn.gov.br/Conteudo.asp?TRAN=ITEM&TARG=240728&ACT=&PAGE=0&PARM=&LBL=ACERVO+DE+MAT%C9RIAS')
    except Exception as e:
        print(e)
    
    soup = BeautifulSoup(url.read(), "html.parser")
    url.close()
    
    url_pdf_link = soup.find_all("div", {"id": "P000"})[0].find_all("ul")

    pattern = r"\d{2}\/\d{2}\/\d{4}"
    for el in url_pdf_link:
        for li in el.find_all("li"):
            try:
                li_date = re.findall(pattern, str(li), re.MULTILINE)[0]
                if li_date == date:
                    return print(li.find_all("a")[0]["href"])
                else:
                    raise FileNotFoundError('Data informada não existe na lista de pdfs.')
            except IndexError:
                pass

In [21]:
def url_from_date(date):
    # Reference date and number to know what's the bulletin number reference
    b_num_ref = 287
    f_date = datetime.date(2021, 1, 1)
    l_date = datetime.date(date[0], date[1], date[2])
    b_num = (l_date - f_date).days
    url = f'http://www.rondonia.ro.gov.br/edicao-{b_num + b_num_ref}-boletim-diario-sobre-coronavirus-em-rondonia/'
    return url


In [22]:
def add_new_row(data_frame, name_row, t_confirmado, t_obito):
    data_frame.loc[-1] = [name_row, t_confirmado, t_obito]  # adding a row
    data_frame.index = data_frame.index + 1  # shifting index
    return data_frame.sort_index()  # sorting by index

In [23]:
dates = ["2021-04-24"]

for date in dates:

    url = url_from_date([int(date) for date in date.split("-")])

    dfs = pd.read_html(url)
    table = dfs[0]

    # Remove two first unecessary lines
    table = table.drop([0, 1]).reset_index(drop=True)

    # Rename columns and remove unecessary columns
    if(table.at[1, 3].lower().startswith('óbitos')):
        table = table[[0, 1, 3]].rename(
                columns={0: "municipio", 1: "confirmados", 3: "mortes"})
    else:
        table = table[[0, 1, 2]].rename(
            columns={0: "municipio", 1: "confirmados", 2: "mortes"})

    # Total values from last row
    last_row_df = table.iloc[-1].name
    total_conf_table = int(table.at[last_row_df, "confirmados"].replace(".", ""))
    total_deaths_table = int(table.at[last_row_df, "mortes"].replace(".", ""))
    
    # Remove unecessary 1 last lines
    table.drop(table.tail(1).index, inplace=True)

    # Create new column with no accents
    table["new"] = table["municipio"].str.normalize('NFKD')\
        .str.encode('ascii', errors='ignore')\
        .str.decode('utf-8')

    # Sort by new column with no accents, then drop
    table = table.sort_values("new", ascending=True)\
        .drop("new", axis=1).reset_index(drop=True)

    # Removing dots and replacing symbols with brasil.io patterns
    for i in range(len(table["confirmados"])):
        table.at[i, "confirmados"] = table.at[i,
                                              "confirmados"].replace(".", "")
        table.at[i, "mortes"] = table.at[i, "mortes"].replace(".", "")
        table.at[i, "municipio"] = table.at[i, "municipio"].replace("’", "\'")

    # Sum total numbers
    total_conf = table["confirmados"].astype(int).sum()
    total_deaths = table["mortes"].astype(int).sum()

    # Adding new row to data_frame with total numbers
    table = add_new_row(table, "Importados/Indefinidos", 0, 0)
    table = add_new_row(table, "TOTAL NO ESTADO",
                        total_conf, total_deaths)

    if total_conf_table == total_conf and total_deaths == total_deaths_table:
        print(date, "Success!!")
    else:
        print(date, "Resultados diferentes ", "Confirmados: ", total_conf,
              total_conf_table, "Mortes: ", total_deaths, total_deaths_table)
        print(date, "Gerado, checagem de resultados primeira página pdf pode apresentar falhas.")

    # Setting a filename
    filename = f"RO_{date}.csv"

    table.to_csv(filename, index=False)


2021-05-27 Success!!
